In [3]:
import pandas as pd

# 엑셀 파일 읽기
file_path = '가격데이터.xlsx'  # 엑셀 파일 경로를 입력하세요.
df = pd.read_excel(file_path)

# 열 이름 리스트 (각 펀드 및 벤치마크)
columns = df.columns[1:]  # 첫 번째 열이 날짜이므로 제외

# 결과를 저장할 데이터프레임 생성
result_df = pd.DataFrame(columns=['date', 'fund_id', 'fund_price', 'benchmark_price'])

# 기준 벤치마크 열 (예: KOSPI200)
benchmark_col = 'KOSPI200'

# 열들을 순회하며 변환
for i, col in enumerate(columns):
    if col == benchmark_col:
        continue  # 벤치마크 열은 별도로 처리
    temp_df = pd.DataFrame({
        'date': df['기준일자'],
        'fund_id': i ,  # fund_id는 1부터 시작
        'fund_price': df[col],
        'benchmark_price': df[benchmark_col]
    })
    result_df = pd.concat([result_df, temp_df], ignore_index=True)

# 결과 출력
print(result_df.head())

# 결과를 CSV로 저장 (옵션)
result_df.to_csv('converted_price_data.csv', index=False)


         date fund_id  fund_price  benchmark_price
0  2024/10/29       1   102438.50          2617.80
1  2024/10/28       1   102559.60          2612.43
2  2024/10/25       1   102521.55          2583.27
3  2024/10/24       1   102413.07          2581.03
4  2024/10/23       1   102393.17          2599.62


C:\Users\rkddk\AppData\Local\Temp\ipykernel_36132\4285633856.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, temp_df], ignore_index=True)


In [36]:
import pandas as pd

# 엑셀 파일 경로
file_path = '포트폴리오 상세화면_데이터.xlsx'  # 실제 파일 경로로 변경

# 시트별 데이터 읽기
fund_info = pd.read_excel(file_path, sheet_name='Sheet1')  # 펀드 정보 시트
asset_composition = pd.read_excel(file_path, sheet_name='Sheet2')  # 투자 비중 시트

# fund_table 생성
fund_table = fund_info[['구분', '운용사', '펀드명', '펀드유형', '투자지역', '1년 수익률', '표준편차', '최대가격', '최소가격','펀드점수','위험도','판매량']]
fund_table.columns = ['fund_id', 'asset_management_company', 'name', 'fund_type', 'investment_area', 'one_year_yield', 'standard_deviation', 'max_price', 'min_price','fund_score','risk_level','sales_volume']

# 수익률과 표준편차를 %로 변환하여 소수점 둘째 자리로 반올림
fund_table['one_year_yield'] = (fund_table['one_year_yield'] * 100).round(2)
fund_table['standard_deviation'] = (fund_table['standard_deviation'] * 100).round(2)
fund_table['fund_score'] = (fund_table['fund_score']).round(2)

# fund_id의 데이터 타입 확인 및 변환
fund_table['fund_id'] = pd.to_numeric(fund_table['fund_id'], errors='coerce')

# asset_composition의 첫 번째 열 데이터 타입 확인 및 변환
asset_composition[asset_composition.columns[0]] = asset_composition[asset_composition.columns[0]].astype(str)

# fund_table의 name 열을 문자열로 변환
fund_table['name'] = fund_table['name'].astype(str)

# asset_composition의 첫 번째 열과 fund_table의 name 열을 병합
asset_composition = asset_composition.merge(
    fund_table[['fund_id', 'name']],
    left_on=asset_composition.columns[0],  # 첫 번째 열(펀드 이름)
    right_on='name',
    how='left'
)

# 'fund_id' 및 필요한 열만 유지
if 'fund_id' in asset_composition.columns:
    asset_composition = asset_composition[['fund_id', '종목명', '비중']]
else:
    print("Error: 'fund_id' column not found in asset_composition")

# 결과 확인
print(fund_table.head())
print(asset_composition.head())

# 데이터 저장 (옵션)
fund_table.to_csv('fund_table.csv', index=False)
asset_composition.to_csv('asset_composition_table.csv', index=False)


Error: 'fund_id' column not found in asset_composition
   fund_id asset_management_company  \
0        1                  케이비자산운용   
1        2                 미래에셋자산운용   
2        3                 키움투자자산운용   
3        4                   한화자산운용   
4        5                 브이아이자산운용   

                                         name fund_type investment_area  \
0             KB RISE 금융채액티브 증권 상장지수 투자신탁(채권)       채권형              국내   
1  미래에셋TIGERCD금리투자KIS특별자산상장지수투자신탁(CD-파생형)(합성)    특별자산파생              국내   
2                키움 KOSEF 단기자금 증권상장지수투자신탁[채권]       채권형              국내   
3              한화 PLUS KOFR금리특별자산상장지수투자신탁(금리)      특별자산              국내   
4            브이아이 뉴굿초이스 우량 단기 증권 투자신탁[채권] C-F       채권형              국내   

   one_year_yield  standard_deviation  max_price  min_price  fund_score  \
0            6.88                1.22  102665.10   99716.80       82.23   
1            3.55                0.05   55151.29   53224.63       82.11   
2            3.76                0

In [37]:
fund_table

,fund_id,asset_management_company,name,fund_type,investment_area,one_year_yield,standard_deviation,max_price,min_price,fund_score,risk_level,sales_volume
0,1,케이비자산운용,KB RISE 금융채액티브 증권 상장지수 투자신탁(채권),채권형,국내,6.88,1.22,102665.10,99716.80,82.23,6,39200
1,2,미래에셋자산운용,미래에셋TIGERCD금리투자KIS특별자산상장지수투자신탁(CD-파생형)(합성),특별자산파생,국내,3.55,0.05,55151.29,53224.63,82.11,5,5958000
2,3,키움투자자산운용,키움 KOSEF 단기자금 증권상장지수투자신탁[채권],채권형,국내,3.76,0.13,104968.90,101278.01,82.02,6,96500
3,4,한화자산운용,한화 PLUS KOFR금리특별자산상장지수투자신탁(금리),특별자산,국내,3.43,0.05,105814.04,102234.59,82.00,6,103700
4,5,브이아이자산운용,브이아이 뉴굿초이스 우량 단기 증권 투자신탁[채권] C-F,채권형,국내,3.98,0.08,1040.91,1004.32,81.98,6,22149
5,6,케이비자산운용,KB RISE 단기 국공채 액티브 증권 상장지수 투자신탁(채권),채권형,국내,3.97,0.24,110850.77,106538.55,81.96,6,203900
6,7,한국투자신탁운용,한국투자 ACE 단기자금 증권 상장지수투자신탁(채권),채권형,국내,3.61,0.12,103664.74,100274.40,81.93,6,121800
7,8,한화자산운용,한화 PLUS 단기채권액티브증권상장지수투자신탁(채권),채권형,국내,4.23,0.37,114772.60,110024.19,81.92,6,38200
8,9,미래에셋자산운용,미래에셋 TIGER 단기채권액티브증권상장지수투자신탁(채권),채권형,국내,3.68,0.12,54710.59,52725.94,81.91,6,622600
9,10,신한자산운용,신한 SOL KIS단기통안채증권상장지수투자신탁[채권],채권형,국내,3.38,0.09,108603.98,104978.31,81.89,6,23800


In [7]:
asset_composition

,fund_id_x,종목명,비중,fund_id_y,name
0,11,엔비디아,22.2,NaN,NaN
1,11,TSMC,13.2,NaN,NaN
2,11,브로드컴,7.9,NaN,NaN
3,11,AMD,4.6,NaN,NaN
4,11,텍사스 인스트루먼트,4.3,NaN,NaN
...,...,...,...,...,...
137,10,주식,0,NaN,NaN
138,10,채권,90,NaN,NaN
139,10,펀드,8.1,NaN,NaN
140,10,유동자산,1.9,NaN,NaN


In [38]:
import pandas as pd

# 펀드 테이블 CSV 읽기
fund_table = pd.read_csv('fund_table.csv')

# 운용사 테이블 생성
asset_management_table = fund_table[['asset_management_company']].drop_duplicates().reset_index(drop=True)
asset_management_table['asset_management_company_id'] = asset_management_table.index + 1  # ID 생성

# 운용사 테이블 CSV 저장 (ID와 운용사 컬럼이 포함됨)
asset_management_table = asset_management_table[['asset_management_company_id', 'asset_management_company']]
asset_management_table.to_csv('asset_management_company.csv', index=False)

# 펀드 테이블에 운용사 ID 추가
fund_table = fund_table.merge(asset_management_table, on='asset_management_company', how='left')
fund_table.to_csv('fund.csv', index=False)


In [29]:
fund_table.to_csv('fund.csv', index=False)

In [24]:
asset_management_table

,asset_managemetn_company,asset_management_company_id
0,케이비자산운용,1
1,미래에셋자산운용,2
2,키움투자자산운용,3
3,한화자산운용,4
4,브이아이자산운용,5
5,한국투자신탁운용,6
6,신한자산운용,7
7,삼성자산운용,8
8,다올자산운용,9


In [28]:
fund_table

,fund_id,asset_management_company,name,fund_type,investment_area,one_year_yield,standard_deviation,max_price,min_price,fund_score,risk_level,sales_volume,asset_management_company_id
0,1,케이비자산운용,KB RISE 금융채액티브 증권 상장지수 투자신탁(채권),채권형,국내,0.0688,0.0122,102665.10,99716.80,82.225964,6,39200,1
1,2,미래에셋자산운용,미래에셋TIGERCD금리투자KIS특별자산상장지수투자신탁(CD-파생형)(합성),특별자산파생,국내,0.0355,0.0005,55151.29,53224.63,82.109457,5,5958000,2
2,3,키움투자자산운용,키움 KOSEF 단기자금 증권상장지수투자신탁[채권],채권형,국내,0.0376,0.0013,104968.90,101278.01,82.020939,6,96500,3
3,4,한화자산운용,한화 PLUS KOFR금리특별자산상장지수투자신탁(금리),특별자산,국내,0.0343,0.0005,105814.04,102234.59,81.996478,6,103700,4
4,5,브이아이자산운용,브이아이 뉴굿초이스 우량 단기 증권 투자신탁[채권] C-F,채권형,국내,0.0398,0.0008,1040.91,1004.32,81.975230,6,22149,5
5,6,케이비자산운용,KB RISE 단기 국공채 액티브 증권 상장지수 투자신탁(채권),채권형,국내,0.0397,0.0024,110850.77,106538.55,81.956884,6,203900,1
6,7,한국투자신탁운용,한국투자 ACE 단기자금 증권 상장지수투자신탁(채권),채권형,국내,0.0361,0.0012,103664.74,100274.40,81.929364,6,121800,6
7,8,한화자산운용,한화 PLUS 단기채권액티브증권상장지수투자신탁(채권),채권형,국내,0.0423,0.0037,114772.60,110024.19,81.920191,6,38200,4
8,9,미래에셋자산운용,미래에셋 TIGER 단기채권액티브증권상장지수투자신탁(채권),채권형,국내,0.0368,0.0012,54710.59,52725.94,81.914075,6,622600,2
9,10,신한자산운용,신한 SOL KIS단기통안채증권상장지수투자신탁[채권],채권형,국내,0.0338,0.0009,108603.98,104978.31,81.886556,6,23800,7


In [41]:
import pandas as pd

# 원본 CSV 파일 읽기
price_data = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.4/Uploads/price_data.csv')

# 변화율 계산 ((금일 가격 - 전일 가격) / 전일 가격) * 100
price_data['price_change_percentage'] = price_data.groupby('fund_id')['fund_price'].pct_change() * 100

# 변화율 컬럼에 NaN 값이 있을 수 있으므로 초기 레코드의 NaN을 0으로 채움
price_data['price_change_percentage'].fillna(0, inplace=True)

# 변경된 데이터프레임을 CSV 파일로 저장
output_file_path = 'price_data_with_changes.csv'  # 저장할 파일 경로
price_data.to_csv(output_file_path, index=False)

print(f"변경된 파일이 '{output_file_path}'에 저장되었습니다.")


변경된 파일이 'price_data_with_changes.csv'에 저장되었습니다.


In [39]:
import pandas as pd

# Customer_Fund 데이터 생성
customer_fund_data = {
    'customer_id': [1, 1, 1, 2, 2, 2],
    'fund_id': [1, 3, 5, 11, 13, 15],
    'investment_percentage': [30.00, 20.00, 50.00, 40.00, 30.00, 30.00],
    'investment_amount': [10000.00, 5000.00, 15000.00, 20000.00, 15000.00, 15000.00],
    'created_at': ['2024-11-18 10:00:00'] * 6
}

# DataFrame 생성
customer_fund_df = pd.DataFrame(customer_fund_data)

# CSV 파일로 저장
customer_fund_df.to_csv('customer_fund.csv', index=False)


In [40]:
customer_fund_df

,customer_id,fund_id,investment_percentage,investment_amount,created_at
0,1,1,30.0,10000.0,2024-11-18 10:00:00
1,1,3,20.0,5000.0,2024-11-18 10:00:00
2,1,5,50.0,15000.0,2024-11-18 10:00:00
3,2,11,40.0,20000.0,2024-11-18 10:00:00
4,2,13,30.0,15000.0,2024-11-18 10:00:00
5,2,15,30.0,15000.0,2024-11-18 10:00:00
